Web scrape world rankings site to pull the top 70 golfers in the world names and then use them as the name in the results table

In [9]:
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
import csv
import pandas as pd

In [2]:
#scrape web page for the name data from http://www.owgr.com/ranking
webpage = requests.get("http://www.owgr.com/ranking")
#store the content in an object
soup = BeautifulSoup(webpage.content, "html.parser")
print(soup)


<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
<link href="/styles/css/font-awesome.min.css" rel="stylesheet"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Official World Golf Ranking - Ranking</title>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="en" http-equiv="content-language"/>
<link href="http://fonts.googleapis.com/css?family=Lato:300,400,700italic" rel="stylesheet" type="text/css"/><link href="/styles/css/normalize.min.css?ov=34" rel="stylesheet" type="text/css"/><link href="/styles/css/mobile.css?ov=34" rel="stylesheet" type="text/css"/><link href="/styles/css/tablet.css?ov=34" rel="stylesheet" type="text/css"/><link hre

In [7]:
#pull all names into a list
names = []
names_array = soup.find_all(attrs={"class":"name"})
#add each name to the names list, ignoring the scroll bar for 0-50 and the header for table in index 51
for name in names_array[52:]:
    text = name.get_text()
    names.append(text)
print(names)
#print(len(names))

['Jon Rahm', 'Collin Morikawa', 'Viktor Hovland', 'Patrick Cantlay', 'Dustin Johnson', 'Rory McIlroy', 'Justin Thomas', 'Xander Schauffele', 'Cameron Smith', 'Hideki Matsuyama', 'Bryson DeChambeau', 'Louis Oosthuizen', 'Jordan Spieth', 'Sam Burns', 'Scottie Scheffler', 'Harris English', 'Abraham Ancer', 'Tony Finau', 'Daniel Berger', 'Brooks Koepka', 'Tyrrell Hatton', 'Billy Horschel', 'Sungjae Im', 'Jason Kokrak', 'Matt Fitzpatrick', 'Patrick Reed', 'Paul Casey', 'Kevin Na', 'Will Zalatoris', 'Webb Simpson', 'Thomas Pieters', 'Joaquin Niemann', 'Talor Gooch', 'Matthew Wolff', 'Marc Leishman', 'Kevin Kisner', 'Max Homa', 'Phil Mickelson', 'Tom Hoge', 'Russell Henley', 'Lee Westwood', 'Corey Conners', 'Tommy Fleetwood', 'Lucas Herbert', 'Harold Varner III', 'Seamus Power', 'Justin Rose', 'Shane Lowry', 'Adam Scott', 'Sergio Garcia', 'Mackenzie Hughes', 'Cameron Tringale', 'Christiaan Bezuidenhout', 'Min Woo Lee', 'Siwoo Kim', 'Richard Bland', 'Ryan Palmer', 'Takumi Kanaya', 'Luke List',

In [10]:
#create / access the database that stores the results table in 
conn = sqlite3.connect("golf_results.db")
cursor = conn.cursor()

In [32]:
#delete the table if it exists
cursor.execute("""DROP TABLE IF EXISTS golf_results""")

#create the table in the database 
cursor.execute("""CREATE TABLE golf_results(
                    id INTEGER PRIMARY KEY,
                    name TEXT NOT NULL,
                    score INTEGER NOT NULL,
                    total INTEGER NOT NULL,
                    round_1 INTEGER NOT NULL,
                    round_2 INTEGER NOT NULL,
                    round_3 INTEGER NOT NULL,
                    round_4 INTEGER NOT NULL,
                    world_ranking INTEGER NOT NULL UNIQUE);""")

In [33]:
#create the list of tuples for the data rows in the table
import random
results = []
i = 1
par = random.randint(70,72)
print("The Course par is " + str(par))
while i <= 100:
    round_1 = random.randint(59,80)
    round_2 = random.randint(59,80)
    round_3 = random.randint(59,80)
    round_4 = random.randint(59,80)
    total = round_1 + round_2 + round_3 + round_4
    score = total - (par*4)
    id_num = i
    name = names[i-1]
    world_ranking = i
    golfer_list = (id_num, name, score, total, round_1, round_2, round_3, round_4, world_ranking)
    results.append(golfer_list)
    i += 1


The Course par is 71


In [34]:
#add list of results to the table
cursor.executemany("""INSERT INTO golf_results VALUES(?,?,?,?,?,?,?,?,?)""", results)
conn.commit()

In [35]:
#print table
df = pd.read_sql_query("""SELECT * FROM golf_results ORDER BY total;""", conn)
print(df)


    id               name  score  total  round_1  round_2  round_3  round_4  \
0   15  Scottie Scheffler    -40    244       59       62       62       61   
1   16     Harris English    -36    248       65       64       60       59   
2   58      Takumi Kanaya    -35    249       64       60       65       60   
3   50      Sergio Garcia    -34    250       59       60       59       72   
4   70    Sebastian Munoz    -30    254       60       62       65       67   
..  ..                ...    ...    ...      ...      ...      ...      ...   
95  28           Kevin Na     18    302       79       80       72       71   
96  93        Adri Arnaus     19    303       78       80       66       79   
97  24       Jason Kokrak     20    304       68       78       79       79   
98  82       Joohyung Kim     21    305       79       80       77       69   
99  47        Justin Rose     25    309       75       78       77       79   

    world_ranking  
0              15  
1          

In [37]:
#print top 10 table 
top_10 = pd.read_sql_query("""SELECT name, score, total, world_ranking FROM golf_results ORDER BY total LIMIT 10;""", conn)
print(top_10)

                 name  score  total  world_ranking
0   Scottie Scheffler    -40    244             15
1      Harris English    -36    248             16
2       Takumi Kanaya    -35    249             58
3       Sergio Garcia    -34    250             50
4     Sebastian Munoz    -30    254             70
5        Shaun Norris    -30    254             73
6      Dean Burmester    -27    257             77
7      Keith Mitchell    -24    260             86
8  Marcus Helligkilde    -24    260             89
9            Tom Hoge    -23    261             39
